In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
!pip install transformers
!pip install pydub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.7 MB/s eta 0:00:00


In [3]:

import torch
import librosa
import numpy as np
import soundfile as sf

from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch

model_id = "facebook/mms-1b-all"

processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)


In [3]:

from pydub import AudioSegment
import os
import librosa
import soundfile as sf

def convert_acc_to_wav(file_path):
    # Check if file is .aac format
    _, file_extension = os.path.splitext(file_path)
    if file_extension.lower() != '.aac':
        print(f'Skipped {file_path} (not an .aac file)')
        return

    # Load .acc file
    audio = AudioSegment.from_file(file_path, format="aac")

    # Convert to .wav
    wav_file_path = file_path.rsplit('.', 1)[0] + '.wav'
    audio.export(wav_file_path, format="wav")

    print(f'Successfully converted {file_path} to {wav_file_path}')

    # Load .wav file
    y, sr = librosa.load(wav_file_path, sr=None)

    # Resample to 16000 Hz
    y_16k = librosa.resample(y, orig_sr=sr, target_sr=16000)

    # Save the new .wav file
    new_wav_file_path = wav_file_path.rsplit('.', 1)[0] + '_16kHz.wav'
    sf.write(new_wav_file_path, y_16k, 16000)

    print(f'Successfully converted {wav_file_path} to 16kHz sample rate at {new_wav_file_path}')
    return new_wav_file_path


# Usage
path = convert_acc_to_wav('/content/drive/MyDrive/FYP/main/eng.aac')


Successfully converted /content/drive/MyDrive/FYP/main/eng.aac to /content/drive/MyDrive/FYP/main/eng.wav
Successfully converted /content/drive/MyDrive/FYP/main/eng.wav to 16kHz sample rate at /content/drive/MyDrive/FYP/main/eng_16kHz.wav


In [4]:
import wave

def check_sample_rate(file_path):
    # Open .wav file
    with wave.open(file_path, 'rb') as wav_file:
        # Get sample rate
        sample_rate = wav_file.getframerate()
        print(f'The sample rate of {file_path} is {sample_rate} Hz')

# Usage
check_sample_rate(path)



The sample rate of /content/drive/MyDrive/FYP/main/eng_16kHz.wav is 16000 Hz


In [5]:


def transcribe_audio(wav_file_path):
    # Load audio
    audio_input, sample_rate = sf.read(wav_file_path)

    inputs = processor(audio_input, sampling_rate=16_000, return_tensors="pt")

    with torch.no_grad():
      outputs = model(**inputs).logits

    ids = torch.argmax(outputs, dim=-1)[0]
    transcription = processor.decode(ids)

    return transcription

# Usage:
transcription = transcribe_audio(path)


print(transcription)

what is the woman wearing


In [6]:
with open ('/content/drive/MyDrive/FYP/main/question.txt', 'w') as file:
    file.write(transcription)

In [8]:

from googletrans import Translator, constants
from pprint import pprint

translator = Translator()

translation = translator.translate("عورت کیا پہن رہی ہے؟", src = "ur", dest="en")
print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")

عورت کیا پہن رہی ہے؟ (ur) --> What is a woman wearing? (en)
